Datenterm: $$\sum_{(i,j)\in k}(r_{ij}-\vec{p}_i^T\vec{q}_j)^2$$
Matrixdarstellung: $$||R-P Q^T||_2^2$$

In [1]:
import numpy as np

def cost(P, Q, R, lambda_, np, nq):
    data_term = np.linalg.norm(R - P @ Q.T)**2
    regularization_p = np.sum(np * np.linalg.norm(P)**2)
    regularization_q = np.sum(nq * np.linalg.norm(Q)**2)
    regularization_term = lambda_ * (regularization_p + regularization_q)
    return data_term + regularization_term

In [2]:
def solveBlock_P(R, Q, I, lambda_, n_p):
    m, n = R.shape
    k = Q.shape[1]
    P = np.zeros((m, k))
    I = I.astype(bool)

    for i in range(m):
        I_i = I[i, :]
        Q_i = Q[I_i, :]
        R_i = R[i, I_i]

        A = Q_i.T @ Q_i + lambda_ * n_p[i] * np.eye(k)
        b = Q_i.T @ R_i
        P[i, :] = np.linalg.solve(A, b)
    return P

def solveBlock_Q(R, P, I, lambda_, n_q):
    m, n = R.shape
    k = P.shape[1]
    Q = np.zeros((n, k))
    I = I.astype(bool)

    for j in range(n):
        I_j = I[:, j]
        P_j = P[I_j, :]
        R_j = R[I_j, j]

        A = P_j.T @ P_j + lambda_ * n_q[j] * np.eye(k)
        b = P_j.T @ R_j
        Q[j, :] = np.linalg.solve(A, b)
    return Q

In [3]:
def calc_rmse(R, I, P, Q):
    R_pred = P @ Q.T
    error = I * (R - R_pred)
    rmse = np.sqrt(np.sum(error**2) / np.sum(I))
    return rmse

def alternating_least_squares(R, I, k=3, lambda_=0.1, max_iter=100, tolerance=1e-4):
    m, n = R.shape
    P = np.random.rand(m, k)
    Q = np.random.rand(n, k)
    n_p = np.ones(m)
    n_q = np.ones(n)
    rmse_history = []

    for iteration in range(max_iter):
        P = solveBlock_P(R, Q, I, lambda_, n_p)
        Q = solveBlock_Q(R, P, I, lambda_, n_q)
        
        rmse = calc_rmse(R, I, P, Q)
        rmse_history.append(rmse)
        
        print(f"Iteration {iteration + 1}/{max_iter}, RMSE: {rmse:.6f}")
        
        if iteration > 0 and abs(rmse_history[-1] - rmse_history[-2]) < tolerance:
            print("convergence reached")
            break

    return P, Q, rmse_history

In [4]:
from utility import setup_dataset, visualize_sparsity_pattern

R, I = setup_dataset(toy_ds=True)
P, Q, rmse_history = alternating_least_squares(R, I, k=3, lambda_=0.1, max_iter=50)
P, Q

Iteration 1/50, RMSE: 0.044326
Iteration 2/50, RMSE: 0.038097
Iteration 3/50, RMSE: 0.035515
Iteration 4/50, RMSE: 0.034847
Iteration 5/50, RMSE: 0.035041
Iteration 6/50, RMSE: 0.035384
Iteration 7/50, RMSE: 0.035695
Iteration 8/50, RMSE: 0.036008
Iteration 9/50, RMSE: 0.036361
Iteration 10/50, RMSE: 0.036768
Iteration 11/50, RMSE: 0.037221
Iteration 12/50, RMSE: 0.037708
Iteration 13/50, RMSE: 0.038217
Iteration 14/50, RMSE: 0.038738
Iteration 15/50, RMSE: 0.039265
Iteration 16/50, RMSE: 0.039791
Iteration 17/50, RMSE: 0.040314
Iteration 18/50, RMSE: 0.040830
Iteration 19/50, RMSE: 0.041338
Iteration 20/50, RMSE: 0.041836
Iteration 21/50, RMSE: 0.042323
Iteration 22/50, RMSE: 0.042799
Iteration 23/50, RMSE: 0.043262
Iteration 24/50, RMSE: 0.043712
Iteration 25/50, RMSE: 0.044149
Iteration 26/50, RMSE: 0.044573
Iteration 27/50, RMSE: 0.044983
Iteration 28/50, RMSE: 0.045380
Iteration 29/50, RMSE: 0.045763
Iteration 30/50, RMSE: 0.046132
Iteration 31/50, RMSE: 0.046489
Iteration 32/50, 

(array([[ 1.69663395, -0.01471549,  1.50488958],
        [ 1.6130473 ,  0.92025477,  1.65421976],
        [ 1.65016947,  0.31767189, -1.16345491],
        [ 1.36917202,  0.3437069 , -0.84398386]]),
 array([[ 1.61664086,  0.01065227,  1.43847342],
        [ 1.27398087,  0.86872786,  1.2411596 ],
        [ 1.89467552,  0.37170304, -1.44588282]]))

In [5]:
R, I = setup_dataset(toy_ds=False)
P, Q, rmse_history = alternating_least_squares(R, I, k=10, lambda_=0.1, max_iter=100)

Iteration 1/100, RMSE: 0.837368
Iteration 2/100, RMSE: 0.750451
Iteration 3/100, RMSE: 0.718357
Iteration 4/100, RMSE: 0.702993
Iteration 5/100, RMSE: 0.694008
Iteration 6/100, RMSE: 0.688124
Iteration 7/100, RMSE: 0.684073
Iteration 8/100, RMSE: 0.681074
Iteration 9/100, RMSE: 0.678723
Iteration 10/100, RMSE: 0.676816
Iteration 11/100, RMSE: 0.675212
Iteration 12/100, RMSE: 0.673845
Iteration 13/100, RMSE: 0.672668
Iteration 14/100, RMSE: 0.671642
Iteration 15/100, RMSE: 0.670743
Iteration 16/100, RMSE: 0.669952
Iteration 17/100, RMSE: 0.669243
Iteration 18/100, RMSE: 0.668592
Iteration 19/100, RMSE: 0.667986
Iteration 20/100, RMSE: 0.667416
Iteration 21/100, RMSE: 0.666873
Iteration 22/100, RMSE: 0.666359
Iteration 23/100, RMSE: 0.665885
Iteration 24/100, RMSE: 0.665452
Iteration 25/100, RMSE: 0.665059
Iteration 26/100, RMSE: 0.664703
Iteration 27/100, RMSE: 0.664377
Iteration 28/100, RMSE: 0.664078
Iteration 29/100, RMSE: 0.663810
Iteration 30/100, RMSE: 0.663565
Iteration 31/100, R